In [353]:
import numpy as np
import pandas as pd
import re

class DataPreprocessor:
    def __init__(self, df):
        self.df = df

    def split_product_info(self, brand_name):
        """브랜드 이름에 따라 제품 정보 분리 및 처리"""
        if brand_name == 'FootHealth':
            self._process_foot_health()
        elif brand_name == 'RedDr':
            self._process_red_dr()
        elif brand_name == 'Xsole':
            self._process_xsole()
        elif brand_name == 'WalkerMaster':
            self._process_walker_master()
        elif brand_name == 'FootMedical':
            self._process_foot_medical()
        elif brand_name == 'Forlax':
            self._process_forlax()
        elif brand_name == 'Sidas':
            self._process_sidas()
        elif brand_name == 'Docsole':
            self._process_docsole()
    '''
    def _process_foot_health(self):
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = splited_txt[0].split(' / ') if len(splited_txt) > 1 else [np.nan, np.nan, np.nan]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            product_step = product_info[0].split(': ')[-1] if len(product_info) > 0 else np.nan
            product_name = product_info[1].split(': ')[-1] if len(product_info) > 1 else np.nan
            product_size = product_info[2].split(': ')[-1] if len(product_info) > 2 else np.nan

            return pd.Series([product_step, product_name, product_size, user_comment])

        self.df[['product_step', 'product_name', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)
    '''

    def _process_foot_health(self):
        # FootHealth 브랜드의 처리 로직
        product_step = []
        product_name = []
        product_size = []
        user_comment = []

        for txt in self.df.product_info:
            splited_txt = txt.split('\n')
            if len(splited_txt) == 2: # 제품정보 + 사용자 코멘트 둘 다 있는 경우
                product = splited_txt[0].split(' / ')
                product_step.append(product[0].split(': ')[-1])
                product_name.append(product[1].split(': ')[-1])
                try: # 사이즈를 입력하지 않은 경우 대비
                    product_size.append(product[2].split(': ')[-1])
                except:
                    product_size.append(np.nan)

                user_comment.append(splited_txt[1])
            else: # 사용자 고멘트만 있는 경우
                product_step.append(np.nan)
                product_name.append(np.nan)
                product_size.append(np.nan)
                user_comment.append(splited_txt[0])
            
        self.df['product_step'] = product_step
        self.df['product_name'] = product_name
        self.df['product_size'] = product_size
        self.df['user_comment'] = user_comment

    def _process_red_dr(self):
        # RedDr 브랜드의 처리 로직
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = re.split('[(): ]', splited_txt[0]) if len(splited_txt) > 1 else [np.nan]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            if len(splited_txt) == 2:
                name = product_info[2] if len(product_info) > 3 else np.nan
                size = product_info[4] if len(product_info) > 5 else product_info[2]
            else:
                size = product_info[2] if len(splited_txt) > 1 else np.nan
                name = np.nan

            return pd.Series([name, size, user_comment])

        self.df[['product_name', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)

    def _process_xsole(self):
        # Xsole 브랜드의 처리 로직
        def parse_product_info(row):
            # 제품, 컬러, 사이즈를 추출하기 위한 정규 표현식 정의
            product_pattern = r'제품: ([^\n/]+)'
            color_pattern = r'컬러: ([^\n/]+)'
            size_pattern = r'사이즈: ([^\n/]+)'

            # 제품, 컬러, 사이즈 정보 추출
            product = re.search(product_pattern, row)
            color = re.search(color_pattern, row)
            size = re.search(size_pattern, row)

            # \n 뒤의 문자열 추출
            user_comment = row.split('\n')[-1] if '\n' in row else ''

            # 각 정보가 존재하면 해당 값을, 아니면 NaN을 반환
            product_name = product.group(1) if product else np.nan
            product_color = color.group(1) if color else np.nan
            product_size = size.group(1) if size else np.nan

            return pd.Series([product_name, product_color, product_size, user_comment])

        # 원본 DataFrame의 'product_info' 열을 사용하여 새로운 컬럼들을 생성
        self.df[['product_name', 'product_color', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_product_info)

    def _process_walker_master(self):
        # WalkerMaster 브랜드의 처리 로직
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = splited_txt[0].split(' / ') if len(splited_txt) > 1 else [splited_txt[0].split()[-1]]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            if len(product_info) >= 2:
                if product_info[0].startswith('사이즈'):
                    size = re.split('[: mm]', product_info[0])[-2]
                    color = re.split(' ', product_info[1])[-1]
                else:
                    size = product_info[1].split()[-1].replace('mm', '')[:7]
                    color = product_info[0].split()[-1].replace('mm', '')
            elif product_info[0].startswith('두족'):
                size = product_info[0].split()[-1].replace('mm', '')[-7:]
                color = product_info[0].split()[-1].replace('mm', '')[:-7]
            else:
                size = np.nan
                color = np.nan

            return pd.Series([color, size, user_comment])

        self.df[['product_color', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)
        
    def _process_foot_medical(self):
        # FootMedical 브랜드의 처리 로직을 간소화한 버전
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = splited_txt[0].split(' / ') if len(splited_txt) > 1 else [np.nan]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            if len(product_info) == 2:
                size = product_info[1].split()[-1][:3]
                name_parts = re.split('[ |(|)]', product_info[0])
                name = name_parts[1] if len(name_parts) > 1 else np.nan
                color = name_parts[2] if len(name_parts) > 2 else np.nan
            else:
                size = splited_txt[0].split()[-1][:3] if len(splited_txt) == 1 else np.nan
                name = np.nan
                color = np.nan

            return pd.Series([name, color, size, user_comment])

        self.df[['product_name', 'product_color', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)

    def _process_forlax(self):
        # Forlax 브랜드의 처리 로직
        product_name = []
        # product_color = []
        product_size = []
        user_comment = []

        for txt in self.df.product_info:
            splited_txt = txt.split('\n')
            if len(splited_txt) == 2: # 제품정보 + 사용자 코멘트 둘 다 있는 경우
                try:
                    product = splited_txt[0].split(' / ')
                    splited_product = product[0].replace(' ', '').replace('1.', ' ').replace('2.', ' ').replace('3.', ' ').replace('4.', ' ').replace('5.', ' ').replace('8.', ' ').replace('mm', '').replace('XS', '(').replace('S', '(').replace('M', '(').replace('L', '(').replace('XL', '(')
                    splited_product = re.split('[:.() ]', splited_product)
                    product_list = [item for item in splited_product if item != '']  # 빈 문자열 요소 제거
                    product_list = [item for item in product_list if item != '0']
                    product_size.append(product_list[2])
                    product_name.append(product_list[1])
                    user_comment.append(splited_txt[1])
                except:
                    product_name.append(np.nan)
                    product_size.append(np.nan)
                    user_comment.append(splited_txt[1])

            else: # 사용자 고멘트만 있는 경우
                # product_color.append(np.nan)
                product_name.append(np.nan)
                product_size.append(np.nan)
                user_comment.append(splited_txt[0])

        self.df['product_name'] = product_name
        self.df['product_size'] = product_size
        self.df['user_comment'] = user_comment

    def _process_sidas(self):
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = re.split(' / ', splited_txt[0]) if len(splited_txt) > 1 else [np.nan]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            step = product_info[0].split()[-1]
            size = re.split('[()]', product_info[1])[-2][:-2]

            return pd.Series([step, size, user_comment])

        self.df[['product_name', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)
    
    def _process_docsole(self):
        def parse_info(row):
            splited_txt = row.split('\n')
            product_info = re.split(' / ', splited_txt[0]) if len(splited_txt) > 1 else [np.nan]
            user_comment = splited_txt[1] if len(splited_txt) == 2 else splited_txt[0]

            if product_info[0].startswith('타입'):
                step = product_info[0].split(': ')[-1]
                size = product_info[1].split()[-1][:-2]
                name = '양발 사이즈 통일'
            else:   
                step = 'N타입(약한 평발)'
                size = product_info[1].split()[-1][:-2] # 우발 사이즈
                name = '양발 사이즈 선택'

            return pd.Series([step, name, size, user_comment])

        self.df[['product_step', 'product_name', 'product_size', 'user_comment']] = self.df['product_info'].apply(parse_info)
       

    def initialize_columns(self):
        self.df['usual_size_mm'] = np.nan
        self.df['narrow_feet'] = False
        self.df['flat_feet'] = False
        self.df['sweaty_feet'] = False
        self.df['finish_quality'] = np.nan
        self.df['size_fit'] = np.nan

    def process_comment(self, comment):
        # 기존의 'process_comment' 함수 내용을 여기에 포함
        sizes = ['210', '215', '220', '225', '230', '235', '240', '245', '250', '255', '260', '265', '270', '275', '280', '285', '290', '295']
        finish_qualities = ['깔끔해', '아주 깔끔해', '엉성해']
        size_fits = ['정', '생각보다 커', '작아']

        splited_comment = re.split("평소사이즈|mm|발|마감처리|예요|요|사이즈", comment)
        filtered_list = [item for item in splited_comment if item != '']  # 빈 문자열 요소 제거

        result = {}
        for e in filtered_list:
            if e in sizes:
                result['usual_size_mm'] = e
            elif e == '칼':
                result['narrow_feet'] = True
            elif e == '평':
                result['flat_feet'] = True
            elif e == '땀 많은 ':
                result['sweaty_feet'] = True
            elif e in finish_qualities:
                result['finish_quality'] = e
            elif e in size_fits:
                result['size_fit'] = e
        return pd.Series(result)

    def apply_process_comment(self):
        self.df.update(self.df['user_comment'].apply(self.process_comment))
        # 'finish_quality' 컬럼의 값 변경
        self.df['finish_quality'] = self.df['finish_quality'].replace({
                                                                        '아주 깔끔해': 'Very Good',
                                                                        '깔끔해': 'Good',
                                                                        '엉성해': 'Poor'
                                                                    })

        # 'size_fit' 컬럼의 값 변경
        self.df['size_fit'] = self.df['size_fit'].replace({
                                                            '정': 'Regular',
                                                            '생각보다 커': 'Larger than Expected',
                                                            '작아': 'Smaller than Expected'
                                                        })

    def find_review_type(self):
        # 기존의 prefix 제거 코드를 여기에 포함
        # NaN 초기화
        self.df['is_best'] = False
        self.df['is_one_month'] = False
        self.df['is_repurchase'] = False

        # 세 가지 종류의 리뷰 유형은 중복 가능, 단어는 '한달사용기' > 'BEST' > '재구매' 순서로 나타남.
        # '한달사용기' prefix 확인 후 해당 단어 제거
        one_month_mask = self.df['review_text'].str.startswith('한달사용기')
        self.df.loc[one_month_mask, 'is_one_month'] = True
        self.df.loc[one_month_mask, 'review_text'] = self.df.loc[one_month_mask, 'review_text'].str.replace('한달사용기', '', 1)

        # 'BEST' prefix 확인 후 해당 단어 제거
        best_mask = self.df['review_text'].str.startswith('BEST')
        self.df.loc[best_mask, 'is_best'] = True
        self.df.loc[one_month_mask, 'review_text'] = self.df.loc[one_month_mask, 'review_text'].str.replace('BEST', '', 1)

        # Check for '재구매' prefix and update columns
        repurchase_mask = self.df['review_text'].str.startswith('재구매')
        self.df.loc[repurchase_mask, 'is_repurchase'] = True
        self.df.loc[one_month_mask, 'review_text'] = self.df.loc[one_month_mask, 'review_text'].str.replace('재구매', '', 1)

    def drop_columns(self):
        self.df = self.df.drop(['product_info', 'user_comment'], axis=1)

    def preprocess(self, brand_name):
        self.split_product_info(brand_name)
        self.initialize_columns()
        self.apply_process_comment()
        self.find_review_type()
        self.drop_columns()


## FootHealth

In [269]:
# FootHealth
df = pd.read_excel('data/FootHealth.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="FootHealth")

In [271]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_step', 'product_name',
       'product_size', 'user_comment', 'usual_size_mm', 'narrow_feet',
       'flat_feet', 'sweaty_feet', 'finish_quality', 'size_fit', 'is_best',
       'is_one_month', 'is_repurchase'],
      dtype='object')

In [272]:
print(df.product_step.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['2단계' '3단계' '1단계' nan]
['액티브(DFH801)' '액티런(DFH803)' '오소틱(DFH902)' '다이나믹(DFH802)' '컴포터(DFH703)'
 '메디워크(DFH903)' '트라이앵글(DFH701)' '메디칼(DFH901)' '서포터(DFH702)' nan]
['270~280' '225~235' '210~220' '255~265' '240~250' '285~295' '300~310' nan
 '225']


In [273]:
df.to_excel('data/prep_data/prep_FootHealth.xlsx', index=False)

## WalkerMaster

In [354]:
df1 = pd.read_excel('data/raw/기성일.xlsx')
df2 = pd.read_excel('data/raw/WalkerMaster.xlsx')
df = pd.concat([df1, df2]).reset_index(drop=True)
df.shape

(5700, 7)

In [355]:
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="WalkerMaster")

In [356]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_color', 'product_size',
       'user_comment', 'usual_size_mm', 'narrow_feet', 'flat_feet',
       'sweaty_feet', 'finish_quality', 'size_fit', 'is_best', 'is_one_month',
       'is_repurchase'],
      dtype='object')

In [357]:
print(df.product_color.unique())
# print(df.product_name.unique())
print(df.product_size.unique())

['블랙' '그레이' '오렌지' '밀리터리' '블루' '핑크' '레드' nan '카오스(블랙)' '스카이(블루)' '더스트(그레이)'
 '루비(레드)' '메이플(오렌지)']
['270~275' '260~265' '250~255' '280~285' '230~235' '240~245' '290~295'
 '220~225' nan '' '230-235' '260-265' '240-245' '250-255' '270-275'
 '290-295' '280-285' '220-225']


In [359]:
df.to_excel('data/prep_data/prep_WalkerMaster.xlsx', index=False)

## FootMedical

In [338]:
# FootHealth
df = pd.read_excel('data/FootMedical.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="FootMedical")

In [339]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_name', 'product_color',
       'product_size', 'user_comment', 'usual_size_mm', 'narrow_feet',
       'flat_feet', 'sweaty_feet', 'finish_quality', 'size_fit', 'is_best',
       'is_one_month', 'is_repurchase'],
      dtype='object')

In [340]:
print(df.product_color.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['회색' '크림' nan]
['원스텝' '사이언스' nan]
['265' '250' '270' '240' '235' '230' '255' '245' '260' '275' '280' '225'
 '285' '290' '210' '220' '215' '노랑' '마감처' '깔끔해' nan '커요']


In [341]:
df.to_excel('data/prep_data/prep_FootMedical.xlsx', index=False)

## Forlax

In [328]:
df = pd.read_excel('data/Forlax.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="Forlax")

In [329]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_name', 'product_size',
       'user_comment', 'usual_size_mm', 'narrow_feet', 'flat_feet',
       'sweaty_feet', 'finish_quality', 'size_fit', 'is_best', 'is_one_month',
       'is_repurchase'],
      dtype='object')

In [330]:
# print(df.product_color.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['하이볼륨' '뉴아치서포트' '베이직' '소프트젤' '솔리드젤' '하이볼륨마그네틱' '아치서포트프로' '소프트젤X' '솔리드젤X'
 nan '하이볼륨자석깔창' '자석깔창' '★풋롤러FOOTRO' '아치서포트' '기능성' '소프트']
['245~270' '275~300' '280~295' '220~240' '235~270' '230~255' '260~275'
 '260~285' '220~235' '240~255' '290~315' nan 'ER★']


In [331]:
df.to_excel('data/prep_data/prep_Forlax.xlsx', index=False)

## RedDr

In [319]:
df = pd.read_excel('data/RedDr.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="RedDr")
# df.head()

In [320]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_name', 'product_size',
       'user_comment', 'usual_size_mm', 'narrow_feet', 'flat_feet',
       'sweaty_feet', 'finish_quality', 'size_fit', 'is_best', 'is_one_month',
       'is_repurchase'],
      dtype='object')

In [321]:
# print(df.product_color.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['안티바이러스' nan '액티비티' 'S' 'M' 'L']
['270~285' '230~245' '250~265' '210~225' nan 'M' 'L' 'S']


In [323]:
df.to_excel('data/prep_data/prep_RedDr.xlsx', index=False)

## Xsole

In [324]:
df = pd.read_excel('data/Xsole.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="Xsole")
# df.head()

In [325]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_name', 'product_color',
       'product_size', 'user_comment', 'usual_size_mm', 'narrow_feet',
       'flat_feet', 'sweaty_feet', 'finish_quality', 'size_fit', 'is_best',
       'is_one_month', 'is_repurchase'],
      dtype='object')

In [326]:
print(df.product_color.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['블루 1.0 ' '블랙 2.0 ' '그레이 ' '화이트 2.0 ' '1) 블랙 ' '1) 블루 ' '2) 블랙 2.0 '
 '1) 블랙 1.0 ' '1.0블랙 ' '1) 화이트 1.0 ' '2) 화이트 ' '블랙 ' '화이트 ' nan '2) 블랙 '
 '1) 블루 1.0 ' '1.0화이트 ' '2.0블랙 ' '1.0블루 ' '2) 화이트 2.0 ' '1.0 블랙 ']
['에어플로우 ' '에어 슬림 ' '2) 에어플로우 2.0 ' '1) 에어플로우 1.0 ' nan]
['265mm' '270mm' '275mm' '290mm' '245mm' '240mm' '260mm' '255mm' '230mm'
 '250mm' '235mm' '260mm~290mm(재단용)' '280mm' '220mm' '230mm~250mm(재단용)'
 '남성용(255~290mm)' '여성용(220~250mm) + 고급포장박스' '여성용(220~250mm)'
 '남성용(255~290mm) + 고급포장박스' '275mm_(차주 발송)' '270mm_(차주 발송)' '280mm_(차주 발송)'
 '275mm(차주 발송)' '240mm(8월 2주차 발송)' '275mm_(차주 출고)' '260mm_(차주 출고)'
 '235mm_(차주 발송)' nan '220mm_(차주 발송)' '260mm(8월 2주차 발송)' '280mm(차주 발송)'
 '250mm_(7월 29일 입고)' '260mm(8월 첫째주 발송 예정)' '240mm_(차주발송)' '240mm(차주 발송)'
 '265mm_(차주 출고)' '290mm_(차주 발송)' '250mm(차주 발송)' '265mm(8월 2주차 발송)'
 '265mm_(차주 발송)' '270mm_(8월 2주차 발송)' '270mm_(차주 출고)' '250mm(8월 2주차 발송)'
 '275mm(8월 첫째주 발송)' '245mm_(7월 29일 입고)' '블랙' '화이트' '265mm (12.09일 발송)'
 '255mm (블랙)' '250mm (블랙)' '260mm

In [327]:
df.to_excel('data/prep_data/prep_Xsole.xlsx', index=False)

## Sidas

In [307]:
df = pd.read_excel('data/Sidas.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="Sidas")
# df.head()

In [308]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_name', 'product_size',
       'user_comment', 'usual_size_mm', 'narrow_feet', 'flat_feet',
       'sweaty_feet', 'finish_quality', 'size_fit', 'is_best', 'is_one_month',
       'is_repurchase'],
      dtype='object')

In [309]:
# print(df.product_color.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['HIGH' 'MID' 'LOW']
['250~265' '235~245' '270~280' '220~230' '285~295' '300~310']


In [310]:
df.to_excel('data/prep_data/prep_Sidas.xlsx', index=False)

## Docsole

In [315]:
df = pd.read_excel('data/Docsole.xlsx')
preprocessor = DataPreprocessor(df)
preprocessor.preprocess(brand_name="Docsole")
# df.head()

In [316]:
df.columns

Index(['review_date', 'user_id', 'rating', 'product_info', 'review_text',
       'image_yn', 'recommendation_count', 'product_step', 'product_name',
       'product_size', 'user_comment', 'usual_size_mm', 'narrow_feet',
       'flat_feet', 'sweaty_feet', 'finish_quality', 'size_fit', 'is_best',
       'is_one_month', 'is_repurchase'],
      dtype='object')

In [317]:
print(df.product_step.unique())
print(df.product_name.unique())
print(df.product_size.unique())

['D타입(정렬은 바르나 아치가 낮은 평발)' 'N타입(약한 평발)' 'B타입(심한 평발)' 'M타입(아치가 높은 요족)'
 'L타입(평발)' 'G타입(아치가 높은 심한 요족)']
['양발 사이즈 통일' '양발 사이즈 선택']
['290' '265' '245' '270' '275' '250' '260' '285' '280' '240' '235' '255'
 '230' '295' '225' '220']


In [318]:
df.to_excel('data/prep_data/prep_Docsole.xlsx', index=False)